In [22]:
import numpy as np
from tensorflow import keras
import tensorflow
from recorder import Recorder
import matplotlib.pyplot as plt
import scipy
import librosa
from librosa import display
from IPython.display import Audio
from scipy.fft import fft, fftfreq
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

In [23]:
!sudo apt-get install libportaudio2


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libportaudio2 is already the newest version (19.6.0-1.1).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.


In [24]:
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libasound2-dev is already the newest version (1.2.6.1-1ubuntu1).
libportaudio2 is already the newest version (19.6.0-1.1).
libportaudiocpp0 is already the newest version (19.6.0-1.1).
portaudio19-dev is already the newest version (19.6.0-1.1).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.


In [12]:
!pip install PyAudio

  Using cached PyAudio-0.2.14.tar.gz (47 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for PyAudio: filename=PyAudio-0.2.14-cp310-cp310-linux_x86_64.whl size=63860 sha256=eabd96e7cdddbe79d204025c2d05b77f70732742aff23d564dcc1e2f417dd8ba
  Stored in directory: /root/.cache/pip/wheels/d6/21/f4/0b51d41ba79e51b16295cbb096ec49f334792814d545b508c5
Successfully built PyAudio


In [14]:
pip install recorder

  Using cached recorder-0.0.2-py3-none-any.whl


In [17]:
frameWidth = 640
frameHeight = 480
cap = cv2.VideoCapture('/content/ambulance.mp4')
#cap = cv2.VideoCapture(1)
cap.set(3, frameWidth)
cap.set(4, frameHeight)
cap.set(10,150)


False

In [18]:
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
np.set_printoptions(suppress=True)
model = tensorflow.keras.models.load_model('/content/keras_model.h5')

In [25]:
def emergency():

   # r = Recorder()
    #r.record(5, output='out.wav')
    fname="/content/sound_1.wav"
    samples, sam_rate=librosa.load(fname, sr=None, mono=True, offset=0.0, duration=None)


    Audio(fname)

    #plt.figure()
    #librosa.display.waveplot(y=samples, sr=sam_rate)
    #plt.xlabel("time secs")
    #plt.ylabel("Ampl")
    #plt.show()

    def fft_plot(audio, sam_rate):
        n = len(audio)
        T= 1/sam_rate
        yf = fft(audio)
        xf = fftfreq(n, T)#[:n//2]
        #fig, ax = plt.subplots()
        #plt.plot(xf, 2.0/n * np.abs(yf)) #np.abs(yf[0:n//2]))
        #plt.grid()
        #plt.xlabel("Freq")
        #plt.ylabel("mag")
        val=np.argmax(yf)
        #plt.show()
        return np.abs(xf[val])


    audio_freq = fft_plot(samples, sam_rate)
    freq = audio_freq.round()
    if freq in range(700,1500):
        #  cv2.putText(image,'Ambulance detecated',(450,100),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),2)

         print("it is a emergency vechicle")
    else:
        pass

In [ ]:
amc = 0
fir = 0
pol = 0
while True:

    success, img = cap.read()
    image = img
    img = cv2.resize(img,(224,224))
    img_res = np.asarray(img)
    normalized_image_array = (img_res.astype(np.float32) / 127.0) - 1
    data[0] = normalized_image_array
    prediction = model.predict(data)
    #print(prediction[0][0])

    #print(text)
    ambulanc_acc =(prediction[0][0])
    firengin_acc = []

    ambulance = (f'ambulance:{prediction[0][0].round()}')
    firengine = (f'firengine:{prediction[0][1].round()}')
    policecar = (f'policecar:{prediction[0][2].round()}')
    trafic =    (f'trafic:{prediction[0][3].round()}')


    if ambulance == 'ambulance:1.0':
        amc = amc + 1
    if amc == 20:
        amc = 0
        emergency()



    if firengine == 'firengine:1.0':
        cv2.putText(image,'Firengine detecated',(450,100),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),2)

    if policecar == 'policecar:1.0':
        cv2.putText(image,'Policecar detecated',(450,100),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),2)

   # if trafic == 'trafic:1.0':
       # cv2.putText(image,'trafic detecated',(50,150),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,0),2)
    print(amc)

    text = (f'{ambulance}% {firengine}% {policecar}% {trafic}% {amc}')
   # print(text)

    cv2.putText(image,text,(15,20),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,0),2)
    cv2_imshow(image)
    if cv2.waitKey(1) and 0xFF == ord('q'):
        break